In [ ]:
#에브리타임 맛집게시판에서 원하는 가게와 관련된 게시물을 크롤링하는 코드입니다.

In [2]:
#!pip install selenium
!pip install kss

     |████████████████████████████████| 65 kB 2.0 MB/s eta 0:00:011


In [3]:
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
import random
from kss import split_sentences
from pandas import DataFrame

In [4]:
df = pd.read_csv("../data/yonsei_dining_list.csv")

In [5]:
df

,dining_id,total_name,dining_name,branch_name,address,latitude,longitude,distance
0,0,이때,이때,NaN,서울특별시 마포구 동교로38안길 7,37.562176,126.924660,1.294640
1,1,대원식당,대원식당,NaN,서울특별시 서대문구 이화여대길 64,37.559250,126.944871,0.920894
2,2,SRS버거킹 이대점,SRS버거킹,이대점,서울특별시 서대문구 이화여대길 30,37.558241,126.945915,1.065890
3,3,벌떼식당,벌떼식당,NaN,서울특별시 마포구 신촌로20길 12,37.554604,126.938169,1.253088
4,4,ZINO본점,ZINO본점,본점,서울특별시 서대문구 연대동문길 45,37.564409,126.944391,0.536427
...,...,...,...,...,...,...,...,...
1923,1923,앵란이모네포차,앵란이모네포차,NaN,서울특별시 서대문구 연세로5가길 16,37.557394,126.935832,0.973146
1924,1924,BARREGGIOSEOUL,BARREGGIOSEOUL,NaN,서울특별시 서대문구 연세로5다길 9,37.556295,126.934908,1.112823
1925,1925,하이 라오상,하이,라오상,서울특별시 서대문구 연세로12길 27,37.559094,126.938546,0.753218
1926,1926,바른치킨 북아현점,바른치킨,북아현점,서울특별시 서대문구 이화여대8길 123,37.563191,126.950283,1.072739


In [6]:
dining_name_list = list(df.dining_name)
dining_name_list

['이때',
 '대원식당',
 'SRS버거킹',
 '벌떼식당',
 'ZINO본점',
 '진국수',
 '고구려',
 '이랜드파크외식신촌그랜드마트',
 '오므토토마토',
 '파주식당',
 '리화인와플',
 '해마로파파이스신촌중앙',
 '예강설렁탕',
 '춘천집닭갈비막국수',
 '최고네',
 '케이즈',
 '그라땅12',
 '코피티암',
 '기꾸초밥',
 '켈리인더볼',
 '신촌형제갈비',
 '도스타코스',
 '향꽃민속주점',
 '도네누이대역점',
 '쥬라기테마파크',
 '황토',
 '지리산삼계탕',
 '후치',
 '쿨노래뱅크',
 'BBQ치킨신촌역점',
 '고주방생라면',
 '지지',
 '초가집삼겹살좋아',
 '강수사',
 '가이스앤돌스',
 '연세대고를샘스낵',
 '가쯔레쯔',
 '깡장국시',
 '종로김밥',
 '거북이집한정식',
 '보통사람분식',
 '나니와',
 '이대치즈밥짱',
 '샌드위치카페',
 '기라꾸',
 '소푸리콩나물국밥',
 '삼청동궁물떡볶이',
 '초당쌈밥',
 '감성매인',
 '다움383',
 '연경',
 '더후라이팬',
 '샤먼갤러리',
 '서브웨이',
 '플러스',
 '신촌서서갈비',
 '멸치국수잘하는집미정국수0410신촌점',
 '웰빙92하우스',
 '구이쟁이꾼',
 '가야가야',
 '스시젠',
 '다꼬야끼',
 '이맛분식',
 '목화씨라운지',
 '훈훈한돼지',
 '희로',
 '레몬',
 '오리지날분식',
 '나나하치',
 '노고산업소명클럽파레스',
 '꾸미면',
 '미도참치',
 '차우삼미',
 '롯데리아',
 '동천홍',
 '뚱스밥버거',
 '제이스틸',
 '제시카키친',
 '하루야',
 '하루키',
 '허프퍼프',
 '프린스치킨',
 '김밥천국연희점',
 '파티메이트',
 '노다메카레',
 '익.coffee',
 '크라제버거',
 '봉설렁탕',
 '무봤나촌닭',
 '남도집',
 '백암순대',
 '통인동커피공방신촌로스팅라이브러리',
 '마린파스타',
 '한솥연희교차로점',
 '코쿤캅',
 '연세대청경관',
 '누룩편주',
 '맛밥김밥

In [7]:
dining_id_list = list(df.dining_id)
total_name_list = list(df.total_name)
branch_name_list = list(df.branch_name)
address_list = list(df.address)
#0~1927

In [9]:
#login
options = wd.ChromeOptions()
options.add_argument("no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent={Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36}")

driver = wd.Chrome("/Users/pdh/TasteMaps/TasteMap/crawler/chromedriver", options=options)

baseURL = 'https://everytime.kr/login'

rand_value = random.uniform(1,2) #랜덤한 시간으로
time.sleep(rand_value) #sleep걸어주기
driver.get(baseURL) #페이지가 넘어갈 때마다 sleep걸어주어서 서버에 무리가 가지 않게 해야함

rand_value = random.uniform(1,2)
time.sleep(rand_value)

driver.find_element_by_name('userid').send_keys("wnwjq462")
driver.find_element_by_name('password').send_keys("ekgus462")

rand_value = random.uniform(1,2)
time.sleep(rand_value)
driver.find_element_by_xpath('//*[@class="submit"]/input').click()

rand_value = random.uniform(1,2)
time.sleep(rand_value)

In [22]:
"""
출력내용에 관하여
0th : dining_name_list에서 0번 id의 검색 내용
링크는 있는데 내용이 없는 경우: 첫 문장에 식당 내용이 없어서 걸러짐
"""

d = dict()
d['dining_id']=[]
d['total_name']=[]
d['dining_name']=[]
d['branch_name']=[]
d['address']=[]
d['rate']=[]
d['comment']=[]
d['source']=[]


for i in dining_id_list[0:1]:
    nextURL = 'https://everytime.kr/378388/all/'+dining_name_list[i]
    driver.get(nextURL)

    rand_value = random.uniform(1,3)
    time.sleep(rand_value)
    
    posts = driver.find_elements_by_css_selector('article>a.article')
    links = [post.get_attribute('href') for post in posts]
    
    for link in links[0:5]:
        driver.get(link)
        
        rand_value = random.uniform(1,3)
        time.sleep(rand_value)
        
        comments = driver.find_elements_by_css_selector('.article>p.large')
        comments_text = comments[0].text
        print(comments_text)
        first_line = (comments_text.split("\n"))[0]
        print("first_line: ", first_line)
        print(i,'th: ','[',link,']')
        for comment in comments:
            print("comment: ", comment.text)
            s_list = split_sentences(comment.text)
            print("s_list 0: ",s_list[0])
            if dining_name_list[i] not in s_list[0]:
                continue
            '''
            if len(s_list)<3:
                if dining_name_list[i] not in s_list[0]:
                    continue
            else:
                if dining_name_list[i] not in s_list[0] or dining_name_list[i] not in s_list[1] or dining_name_list[i] not in s_list[2]:
                    continue
            '''
            d['dining_id'].append(i)
            d['dining_name'].append(dining_name_list[i])
            d['total_name'].append(total_name_list[i])
            d['branch_name'].append(branch_name_list[i])
            d['address'].append(address_list[i])
            d['rate'].append(None)
            d['comment'].append(comment.text)
            d['source'].append('에브리타임')
            
            print(comment.text)
        print()

data_df = DataFrame(d)
#data_df.to_csv({wheretosave}, index=False, encoding="utf-8-sig")

백양누리 더라운지는 후기가 없네?
first_line:  백양누리 더라운지는 후기가 없네?
0 th:  [ https://everytime.kr/378388/v/161247375 ]
comment:  백양누리 더라운지는 후기가 없네?
s_list 0:  백양누리 더라운지는 후기가 없네?

하니앤손스(첫번째글) 쓴 사람인데
여기도 좋아요!!

손님이 많이 없으셔서 사장님이 많이 슬퍼하시는 것 같아요....ㅠ

1. 메뉴

오미자요거트스무디 (6.0)
아쌈밀크티 (5.5)
당근케이크 (6.0)

2. 맛과 친절

밀크티 덕훈데 아쌈 한 종류 밖에 없어서ㅜ
슬퍼하며 시켰는데 웬걸... 넘 맛있음 당도 물어봐주시는데 적당히 달게 부탁드려용 하니까
딱 당에 취하지 않으면서 기분좋게 달달하고 아쌈향이 한 모금마다 기분좋게 퍼지는.....츄릅 이건 그냥 내가 밀크티 덕후여서 그런 걸수도💁🏻‍♀ㅎㅎ..

오미자요거트스무디는 오미자보다 요거트 맛이 강했음
내가 오미자가 무슨 맛인지 잘 몰라서ㅋㅋ 감별을 잘 못 한거 같기도 함

당근케이크도 짱맛임🥕🍰 물론 이것도 내가 덕후지만...
가끔 당근케이크를 배달로 시켜보면 그 천편일률적인 맛이 있는데 (같은 납품업체 제품인 듯..ㅠ)
그거보다 덜 달고 당근은 좀 더 많고
건포도같이 생겼지만 먹어보니 견과류인(??) 것도 있고
dalla dalla... 하고 있는데 나중에 저희가 직접 만들어서 그래여.....하고 알려주셨음

앗 그리고... 이때가 두번째 방문이었는데
이틀전에 온 거 기억해 주시고
지난번에 쿠폰 찍으셨어요? 쿠폰 하실래요? 하시면서 이틀 전에 시킨 것까지...! 도장 찍어주셨다..ㅠㅠ♥️

3. 결론
(커피맛은 잘 모르지만...)
맛있는 차와 케이크(티라미수도 있드랑)를 냠냠하며 과제 or 데이트하고 싶다! 추천!! 그냥 먹으러 가도 추천!!!!
first_line:  하니앤손스(첫번째글) 쓴 사람인데
0 th:  [ https://everytime.kr/378388/v/147243845 ]
comment:  하니앤손스

In [ ]:
# reference
# https://yeoulcoding.tistory.com/121
# https://velog.io/@psr2110/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B81%EC%A3%BC%EC%B0%A8-%ED%81%AC%EB%A1%A4%EB%A7%81